In [1]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('../data/2022_07_13_WTvsKO_daaRSL3_pval_subset.csv',header=2)

In [6]:
data

,Compound,Neutral mass (Da),m/z,Charge,Retention time (min),CCS (angstrom^2),Chromatographic peak width (min),Identifications,Anova (p),q Value,...,Accepted Description,Adducts,Formula,Score,Fragmentation Score,Mass Error (ppm),Isotope Similarity,Retention Time Error (mins),dCCS (angstrom^2),Compound Link
0,1.09_838.6328m/z,NaN,838.632769,1,1.086450,280.775911,0.222850,0,0.000004,0.020826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.24_281.3243m/z,NaN,281.324318,1,1.241600,178.633257,0.397767,0,0.000022,0.053786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.33_610.4790n,610.479009,647.462248,1,1.329067,247.259784,1.551633,0,0.000033,0.054200,...,NaN,"M-H2O-H, M+Na-2H, M+Cl, M+K-2H",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.36_877.6909n,877.690898,912.659715,1,6.362000,299.261536,0.276450,0,0.000065,0.065990,...,NaN,"M-H2O-H, M+Cl",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.14_810.6040m/z,NaN,810.604026,1,1.140067,280.932758,0.310317,0,0.000067,0.065990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,3.38_875.4953m/z,NaN,875.495285,1,3.380200,296.308232,0.586767,0,0.049541,0.385791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630,4.35_707.0502n,707.050187,744.033132,1,4.345000,211.722671,1.328733,0,0.049632,0.385884,...,NaN,"M+Cl, M+K-2H",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,1.61_1160.7376n,1160.737637,1195.763923,1,1.605533,357.528331,0.620700,0,0.049730,0.386039,...,NaN,"M-H2O-H, M+Cl",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
632,6.73_619.4305m/z,NaN,619.430485,1,6.725917,241.137295,0.378017,0,0.049873,0.386187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
retention_time = data['Retention time (min)'][:].to_numpy().astype('float32')
ccs = data['CCS (angstrom^2)'][:].to_numpy().astype('float32')
mz = data['m/z'][:].to_numpy().astype('float32')
compound_name = data['Compound'][:].to_numpy().astype('str')

In [10]:
# m/z conditions
truth_ls = []
for i in range(len(mz)):
  if i%100 == 0:
    print(i)
  mz_new = np.delete(mz,i)
  for j in range(len(mz_new)):
    if (mz_new[j] <= mz[i] + 6.04 and mz_new[j] >= mz[i]+6.02) or (mz_new[j] >= mz[i] - 6.04 and mz_new[j] <= mz[i]-6.02):
      truth_ls.append([i,j])

0
100
200
300
400
500
600


In [16]:
len(truth_ls)

120

In [17]:
# Retention time conditions
truth_ls_2 = []
for i in range(len(truth_ls)):
  idx1 = truth_ls[i][0]; idx2 = truth_ls[i][1]
  rt1 = retention_time[idx1]
  retention_time_new = np.delete(retention_time,idx1)
  rt2 = retention_time_new[idx2]
  if rt1 <= rt2+rt2*.03 and rt1 >= rt2-rt2*.03:
    truth_ls_2.append(truth_ls[i])


In [18]:
len(truth_ls_2)

42

In [ ]:
# CCS conditions
truth_ls_3 = []
for i in range(len(truth_ls_2)):
  idx1 = truth_ls_2[i][0]; idx2 = truth_ls_2[i][1]
  rt1 = ccs[idx1]
  ccs_new = np.delete(ccs,idx1)
  rt2 = ccs_new[idx2]
  if rt1 <= rt2+rt2*.05 and rt1 >= rt2-rt2*.05:
    truth_ls_3.append(truth_ls_2[i])


In [ ]:
len(truth_ls_3)

In [ ]:
for i in range(len(truth_ls_3)):
  idx1 = truth_ls_3[i][0]; idx2 = truth_ls_3[i][1]
  str1 = compound_name[idx1]
  compound_new = np.delete(compound_name,idx1)
  str2 = compound_new[idx2]
  print('Compound 1: ' + str1 + ' paired with Compound 2:' + str2)

In [ ]:
print(truth_ls_3)

In [ ]:
  mass1_list = []
  mass2_list = []
  rt1_list = []
  rt2_list = []
for i in range(len(truth_ls_3)):
   idx1 = truth_ls_3[i][0]; idx2 = truth_ls_3[i][1]
   mass1 = mz[idx1]
   ret1 = retention_time[idx1]
   mass_new = np.delete(mz,idx1)
   mass2 = mass_new[idx2]
   ret_new = np.delete(retention_time, idx1)
   ret2 = ret_new[idx2]
   mass1_list.append(mass1)
   mass2_list.append(mass2)
   rt1_list.append(ret1)
   rt2_list.append(ret2)
outputdata = {'Mass 1': mass1_list, 'RT 1': rt1_list, 'Mass 2': mass2_list, 'RT 2': rt2_list}
outputdf = pd.DataFrame(outputdata, columns = ['Mass 1', 'RT 1', 'Mass 2', 'RT 2'])
outputdf.head()
outputdf.to_excel('../data/output.xlsx')